In [189]:
import pandas as pd
import os
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [190]:
HOME = os.path.expanduser("~") + "/"

In [191]:
DIR = HOME + "Desktop/paperbase_data/preprocessed"

In [192]:
allfiles = glob.glob(os.path.join(DIR,"*.jpg"))

In [193]:
df = pd.DataFrame({"localpath":allfiles})

In [194]:
rf = pd.read_csv(HOME + "Desktop/paperbase_data/genome_roughness_1365_250_201_5_subtract.csv")

In [195]:
rf['pathstring'] = ["_".join(item.split("/"))[:-4] + ".jpg" for item in rf.localpath]

In [196]:
df['pathstring'] = [os.path.basename(item) for item in df.localpath]

In [197]:
df = df.join(rf[['pathstring','roughness']].set_index('pathstring'),"pathstring")

In [198]:
#df.to_csv("genome_roughness_all.csv",index=False)

In [199]:
set(["_".join(item.split("_")[:7]) for item in df.pathstring])

{'_Volumes_G-DRIVE Thunderbolt 3_genome_texture_LML Collection_texture'}

In [200]:
df['pathstring'] = ["_".join(item.split("_")[7:]) for item in df.pathstring]

In [201]:
set([item[-4:] for item in df.pathstring])

{'.jpg'}

In [202]:
tmp = [item[:-4] for item in df.pathstring]

In [203]:
set([len(item.split("_")) for item in tmp])

{5, 6}

In [207]:
[item for item in tmp if len(item.split("_"))==6]

['binder4_jd_4213_mt_r_2',
 'binder4_jd_4213_mt_r_1',
 'binder4_jd_4213_mt_r_0',
 'binder7_jd_1844__im_2',
 'binder7_jd_1844__im_0',
 'binder7_jd_1844__im_1',
 'box1.1_jd_940k_kc_c_2',
 'box1.1_jd_904z_kv_c_1',
 'box1.1_jd_904z_kv_c_0',
 'binder4_jd_4213_mt_y_2',
 'binder4_jd_4213_mt_b_2',
 'box1.1_jd_940k_kc_c_1',
 'binder4_jd_4213_mt_b_0',
 'binder4_jd_4213_mt_y_0',
 'box1.1_jd_904z_kv_c_2',
 'binder4_jd_4213_mt_y_1',
 'binder4_jd_4213_mt_b_1',
 'box1.1_jd_904x_kc_c_0',
 'box1.1_jd_904x_kc_c_1',
 'box1.1_jd_904x_kc_c_2']

In [208]:
df['catalog'] = [item.split("_")[2] for item in tmp]

In [210]:
badlist = ['_r_','_y_','_c_','_b_']

In [212]:
for item in badlist:
    df = df[~df.pathstring.str.contains(item)]

In [215]:
len(df.catalog.unique())

5586

In [216]:
rows = []
for catalog in df.catalog.unique():
    tmp = df[df.catalog==catalog]
    median_roughness = np.median(tmp.roughness)
    tmp['difference_from_median'] = tmp.roughness.apply(lambda x: abs(median_roughness - x))
    tmp = tmp[tmp.difference_from_median == tmp.difference_from_median.min()].iloc[0]
    rows.append(tmp)

In [217]:
df = pd.DataFrame(rows)

In [218]:
df.catalog.value_counts().head()

2378     1
4913d    1
1684     1
2211h    1
5515v    1
Name: catalog, dtype: int64

In [220]:
try:
    os.mkdir("texture")
except:
    pass

In [221]:
import shutil

In [222]:
for i in df.index:
    catalog = df.catalog.loc[i]
    localpath = df.localpath.loc[i]
    savestring = "texture/" + catalog + ".jpg"
    shutil.copy(localpath,savestring)

In [225]:
df = df[['catalog','roughness']]

In [227]:
df.to_csv("texture.csv",index=False)